# Классификация заголовков документов
Есть файл с заголовками документов и классами этих документов.  
Задача - построить алгоритм, который по этим данным мог бы по возможности однозначно классифицировать заголовки документов похожие на те, которые есть в имеющемся файле.  
После предварительного просмотра заголовков есть предположение, что заголовки имеют примерно один вид и отличаются деталями формулировок и ключевыми словами. Однако в заголовках также есть "мусор", который будет мешать нормальной классификации.  
В итоге надо создать алгоритм, который бы определял характерные для каждого класса ключевые слова, которые были бы реально значимыми.  

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
from nltk.corpus import stopwords
from pymystem3 import Mystem
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.tree import DecisionTreeClassifier, plot_tree


RANDOM_STATE = 423

# Список стоп-слов
russian_stopwords = stopwords.words("russian")
russian_stopwords.extend([' ', '\n'])

# Лемматизатор
lemmatizer = Mystem()

# Словарь опесаток и исправлений в пормате {"опечатка": "корректное значение"}
typos = {}


## Предобработка данных

### Загрузка тренировочных данных

In [2]:
df_titles_raw = pd.read_csv("../data/documents_titles.csv", sep=";")
df_titles = df_titles_raw.copy()
df_titles.iloc[0,0] = df_titles.iloc[0,0].replace('\ufeff', '')
df_titles.head()

,document_type_code,document_title
0,bills,"Список объектов, по которым решением общего со..."
1,bills,"Список объектов, по которым решением общего со..."
2,bills,"Список объектов, по которым решением общего со..."
3,bills,"Список объектов, по которым решением общего со..."
4,bills,"Список объектов, по которым решением общего со..."


Все заголовки документов имеют примерно один вид, но отличаются деталями формулировки. Предполагаю, что для успешной классификации будет достаточно определить списки ключевых слов, характерных для конкретного типа документов. Тогда, прежде чем классифицировать документы, из их заголовков надо убрать знаки пунктуации, числовые значения (года, даты) и незначащие слова.

### Вызов пользовательского удаления данных
Может возникнуть необходимость удаления из текста каких-то заранее известных фрагментов.  
Для этого предусмотрен вызов функции удаления, который может быть определен пользователем.  
В этом наборе данных это довольно простая замена текста на пустую строку, но могут быть и другие, более сложные алгоритмы.  

In [3]:
def castom_text_clear(input_text: str):
    return re.sub(r'по\s+адрес(у|ам)\:\s+АДРЕСА?\s+СКРЫТЫ?', '', input_text)
    

### Предварительная чистка от пунктуации, цифр и т. д.

In [4]:
def remove_symbols_and_empty_words(input_text: str):
    result = input_text.lower()
    
    # Удаление цифр и знаков препинания
    result = re.sub(r'[^\w\s]|\d', '', result)
    
    # удаление слов типа "год", "месяц" и т.п.
    result = re.sub(r'\bгод(а|ы|у|ом){0,1}\b', '', result)
    result = re.sub(r'\bмесяц(а|ы|е){0,1}\b', '', result)
    
    # удаление названий месяцев
    month_template = (r'\b(январ[ьяе])|(феврал[ьяе])|(март[ае]?)' 
                      + '|(апрел[ьяе])|(ма[йяе])|(июн[ьяе])|(июл[ьяе])' 
                      + '|(август[ае]?)|(сентябр[ьяе])|(октябр[ьяе])' 
                      + '|(ноябр[ьяе])|(декабр[ьяе])')
    result = re.sub(month_template, '', result)

    # Замена переноса строки и табуляции на пробелы
    result = re.sub(r'\n|\t', ' ', result)
    result = re.sub(r'\s{2,}', ' ', result)

    result = result.strip()
    return result


### Проведение лемматизации и чистка от стоп-слов

In [5]:
def title_lemmatize_and_clear(input_text: str, stop_words: list):
    return ' '.join([w for w in lemmatizer.lemmatize(input_text) 
                     if w not in stop_words])


### Вызов исправления ошибок и опечаток.
Здесь пользователь может задать словарь заранее известных опечаток и ошибок в словах с корректными значениями для их замены.  

In [6]:
def correct_typos(input_text: str, typos_dict: dict):
    result = input_text
    for typo in typos_dict.keys():
        result = result.replace(typo, typos_dict[typo])
    return result

### Весь цикл предобработки

In [7]:
def title_preprocessing(input_text: str):
    """
    Полный цикл предобработки текста заголовка документа
    """
    result = input_text
    # 2. Вызов пользовательского удаления данных
    result = castom_text_clear(result)
    # 3. Предварительная чистка от пунктуации, цифр и т.д.
    result = remove_symbols_and_empty_words(result)
    # 4. Проведение лемматизации и чистка от стоп-слов
    result = title_lemmatize_and_clear(result, russian_stopwords)
    # 5. Вызов исправления ошибок и опечаток
    result = correct_typos(result, typos)
    
    return result

### Обработка заголовков

In [8]:
%%time
df_titles.document_title = df_titles_raw.document_title.apply(title_preprocessing)

CPU times: user 741 ms, sys: 45.3 ms, total: 786 ms
Wall time: 2.77 s


## Получение признаков из заголовков документов

### Формирование матрицы слов

In [9]:
def get_words_tfidf():
    '''
    Получение матрицы значений TF-IDF по словам их заголовков документов в виде DataFrame
    '''
    tfidf = TfidfVectorizer()
    tfidf_matrix = tfidf.fit_transform(df_titles["document_title"])
    return pd.DataFrame(tfidf_matrix.toarray(), index=df_titles.index, columns=tfidf.get_feature_names_out())


df_words = get_words_tfidf()
df_words.head()

,xlsx,ведение,взнос,владелец,владение,влфделец,выявлять,гоплюс,дважды,день,...,учитывать,фактический,формировать,формироваться,хозяйственный,хотя,частично,часть,экз,являться
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.294927,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.294927,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.294927,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.294927,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.294927,0.0,0.0,0.0,0.0,0.0,0.0


In [10]:
print(list(df_words.columns[df_words.sum()>0]))

['xlsx', 'ведение', 'взнос', 'владелец', 'владение', 'влфделец', 'выявлять', 'гоплюс', 'дважды', 'день', 'должный', 'дооплата', 'доплата', 'допускать', 'задолженность', 'идентификатор', 'итоговый', 'количество', 'корректировка', 'корректность', 'который', 'кроме', 'находиться', 'начисление', 'неккоректно', 'неккоректный', 'некорректный', 'некорретной', 'необходимо', 'необходимость', 'неотозванный', 'непривязанный', 'образовываться', 'общий', 'объект', 'объем', 'оплата', 'оплачивать', 'оплюсганизация', 'определять', 'организаци', 'организация', 'осуществленоосуществлять', 'осуществлять', 'отмена', 'отменять', 'отношение', 'отражение', 'ошибочно', 'пеплюседача', 'передача', 'переиод', 'переплата', 'перераспределение', 'перерплата', 'перечень', 'период', 'письмо', 'плательщик', 'погашение', 'поле', 'превышение', 'предыдущий', 'признание', 'признанный', 'приложение', 'принадлежать', 'причина', 'производить', 'ранее', 'располагать', 'расположенный', 'рассмотрение', 'резерв', 'решение', 'свя

### Обнаруженные ошибки и опечатки

'влфделец' -> 'владелец'  
'гоплюс' -> 'год'  
'оплюсганизация' ->   
'пеплюседача' -> 'передача'  
'стоплюсонный' -> 'сторонний'  
'организаци' ->   
'осуществленоосуществлять' ->   
'переиод' -> 'период'  
'перерплата' -> 'переплата'  
'неккоректно' -> 'некорретной'  
'неккоректный' -> 'некорректный'  
  
'декабрь' - почему-то не удалился  
'подекабрь' -> 'по декабрь' -> удалить  
 
'экз' - какой-то мусор  -> удалить  

### Заполнение словаря ошибок и повторная обработка
Здесь заполняется словарь опечаток `typos`, который используется в функции предобработки данных `title_preprocessing()`.

In [11]:
typos.update({'влфделец': 'владелец',
              'гоплюс': '',
              'гоплюсе': '', 
              'оплюсганизация': 'организация',
              'пеплюседача': 'передача',
              'стоплюсонный': 'сторонний',
              'организаци': 'организация',
              'осуществленоосуществлять': 'осуществлять',
              'переиод': 'период',
              'перерплата': 'переплата',
              'неккоректно': 'некорретной',
              'неккоректный': 'некорректный',
              'подекабрь': '',
              'экз': '',})

### Повторная предобработка данных с учетом опечаток

In [12]:
%%time
df_titles.document_title = df_titles_raw.document_title.apply(title_preprocessing)

CPU times: user 751 ms, sys: 58.7 ms, total: 809 ms
Wall time: 2.2 s


In [13]:
df_words = get_words_tfidf()
df_words.head()

,xlsx,ведение,взнос,владелец,владение,выявлять,дважды,день,должный,дооплата,...,учет,учитывать,фактический,формировать,формироваться,хозяйственный,хотя,частично,часть,являться
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.294942,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.294942,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.294942,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.294942,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.294942,0.0,0.0,0.0,0.0,0.0


In [14]:
print(list(df_words.columns))

['xlsx', 'ведение', 'взнос', 'владелец', 'владение', 'выявлять', 'дважды', 'день', 'должный', 'дооплата', 'доплата', 'допускать', 'задолженность', 'идентификатор', 'итоговый', 'количество', 'корректировка', 'корректность', 'который', 'кроме', 'находиться', 'начисление', 'некорректный', 'некорретной', 'необходимо', 'необходимость', 'неотозванный', 'непривязанный', 'образовываться', 'общий', 'объект', 'объем', 'оплата', 'оплачивать', 'определять', 'организация', 'организацияя', 'осуществлять', 'отмена', 'отменять', 'отношение', 'отражение', 'ошибочно', 'передача', 'переплата', 'перераспределение', 'перечень', 'период', 'письмо', 'плательщик', 'погашение', 'поле', 'превышение', 'предыдущий', 'признание', 'признанный', 'приложение', 'принадлежать', 'причина', 'производить', 'ранее', 'располагать', 'расположенный', 'рассмотрение', 'резерв', 'решение', 'связь', 'скрывать', 'собрание', 'состояние', 'специализированный', 'специальный', 'список', 'сторона', 'сторонний', 'сумма', 'сформировывать

### Формирование списка значащих слов

In [15]:
feature_columns_names = list(df_words.columns[df_words.sum() > 0])

## Обучение модели
Для классификации я решил использовать семейство бинарных классификаторов на решающих деревьях, для каждого класса своё дерево.
Общий результат классификации будет определяться так:
- Если заголовок был классифицирован как положительный класс только одним из решающих деревьев, то считается, что классификация прошла успешно и рассматриваемый заголовок относится к этому классу.
- Если ни одно решающее дерево не классифицировало заголовок как относящийся к положительному классу, то считается, что данный заголовок не знаком модели, и возвращается пустое значение класса.
- Если несколько решающих деревьев определили заголовок как относящийся к положительному классу, то формируется и возвращается список этих классов, классификация считается неуспешной.

Во всех случаях, когда классификация была неуспешной, нужно сообщить пользователю об этом для дальнейшего изучения причин возникновения такой ситуации. 

### Формирование списка типов документов

In [16]:
document_types = df_titles.document_type_code.str.strip().unique()

### Формирование признаков

In [17]:
def get_titles_data(feature_columns: list):
    '''
    Получение DataFrame с признаками и целевой переменной для заголовков документов
    '''
    words_df = get_words_tfidf()
    if feature_columns:
        words_df = words_df[feature_columns]
    return df_titles.join(df_words).drop(columns=["document_title"])


df_titles_data = get_titles_data(feature_columns_names)
df_titles_data.head()

,document_type_code,xlsx,ведение,взнос,владелец,владение,выявлять,дважды,день,должный,...,учет,учитывать,фактический,формировать,формироваться,хозяйственный,хотя,частично,часть,являться
0,bills,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.294942,0.0,0.0,0.0,0.0,0.0
1,bills,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.294942,0.0,0.0,0.0,0.0,0.0
2,bills,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.294942,0.0,0.0,0.0,0.0,0.0
3,bills,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.294942,0.0,0.0,0.0,0.0,0.0
4,bills,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.294942,0.0,0.0,0.0,0.0,0.0


### Обучение списка бинарных классификаторов

In [18]:
X = df_titles_data.drop("document_type_code", axis=1)

classifiers_list = []
for dtc in document_types:
    clf = DecisionTreeClassifier(random_state=RANDOM_STATE)
    y = (df_titles_data.document_type_code == dtc).astype(int)
    clf.fit(X, y)
    clf_score = clf.score(X, y)
    classifiers_list.append((clf, dtc, clf_score))

    print(f"{dtc}:\t{clf_score}")
print("===================================")
print(f"Average score = {sum([x[2] for x in classifiers_list]) / document_types.shape[0]}")


bills:	1.0
bills-corrections-reduction:	1.0
bills-corrections-old:	0.9986225895316805
bills-corrections-new:	0.9986225895316805
bills-without-title-16:	1.0
payments:	1.0
payments-additional:	0.9931129476584022
payments-over-accounting:	1.0
payments-over-incorrect:	1.0
payments-correct:	1.0
payments-over:	1.0
payments-over-accounting-cancellation:	0.9931129476584022
payments-over-accounting-cancellation-incorrec:	1.0
bills-corrections-reduction-due-to-non-payment:	1.0
payments-initial:	1.0
bills-initial:	1.0
bills-corrections-initial:	1.0
bills-corrections-reduction-cancellation:	1.0
Average score = 0.9990817263544535


По итогам классификации видно, что не все заголовки были классифицированы однозначно. Эти случаи нужно исследовать и сообщить составителям классифицируемых документов, чтобы совместно выработать решения для однозначной классификации этих документов. Это может быть как внесение дополнительных признаков в источник данных, так и корректировка самих документов. 

### Получение значимых признаков из обученных классификаторов

In [19]:
usefull_features = set()
for clf, _, _ in classifiers_list:
    usefull_features.update(clf.feature_names_in_[clf.feature_importances_ > 0])
feature_columns_names = sorted(usefull_features)
print(f"Количество значимых признаков: {len(feature_columns_names)}")
print(feature_columns_names)

Количество значимых признаков: 47
['дооплата', 'доплата', 'допускать', 'задолженность', 'корректность', 'который', 'кроме', 'находиться', 'начисление', 'необходимо', 'необходимость', 'неотозванный', 'непривязанный', 'объект', 'оплата', 'определять', 'организация', 'организацияя', 'осуществлять', 'отмена', 'отменять', 'ошибочно', 'переплата', 'перечень', 'период', 'плательщик', 'погашение', 'превышение', 'признание', 'принадлежать', 'производить', 'ранее', 'располагать', 'связь', 'состояние', 'специализированный', 'список', 'также', 'увеличение', 'указывать', 'уменьшение', 'учет', 'учитывать', 'формировать', 'формироваться', 'хотя', 'частично']


В итоге видно, что реально используемых моделью признаков не так много. Этот список имеет смысл зафиксировать и повторно построить модель только на этих признаках.

### Обучение модели с ограниченным списком признаков

In [20]:
df_titles.document_title = df_titles_raw.document_title.apply(title_preprocessing)

df_titles_data = get_titles_data(feature_columns_names)

X = df_titles_data.drop("document_type_code", axis=1)

classifiers_list = []
for dtc in document_types:
    clf = DecisionTreeClassifier(random_state=RANDOM_STATE)
    y = (df_titles_data.document_type_code == dtc).astype(int)
    clf.fit(X, y)
    clf_score = clf.score(X, y)
    classifiers_list.append((clf, dtc, clf_score))

    print(f"{dtc}:\t{clf_score}")
print("===================================")
print(f"Average score = {sum([x[2] for x in classifiers_list]) / document_types.shape[0]}")

bills:	1.0
bills-corrections-reduction:	1.0
bills-corrections-old:	0.9986225895316805
bills-corrections-new:	0.9986225895316805
bills-without-title-16:	1.0
payments:	1.0
payments-additional:	0.9931129476584022
payments-over-accounting:	1.0
payments-over-incorrect:	1.0
payments-correct:	1.0
payments-over:	1.0
payments-over-accounting-cancellation:	0.9931129476584022
payments-over-accounting-cancellation-incorrec:	1.0
bills-corrections-reduction-due-to-non-payment:	1.0
payments-initial:	1.0
bills-initial:	1.0
bills-corrections-initial:	1.0
bills-corrections-reduction-cancellation:	1.0
Average score = 0.9990817263544535


## Итоги исследования и дальнейшие шаги

По итогам исследования имеющихся данных была выработана структура модели машинного обучения, отвечающая требованиям поставленной задачи. Теперь требуется реализовать эту модель в виде отдельного класса с возможностью:
- обучения модели,
- сохранения модели в файл,
- загрузки модели из файла,
- получения результата классификации новых заголовков документов.

Кроме того, нужно будет проверить качество полученной модели на неизвестных ей данных.